In [25]:
import os
import cv2
import numpy as np
from typing import List

test_images_path = 'human_in_the_loop/train'
output_path = 'augmented'

#### Apply mirror data augmentation on livecell_test_images/A172

In [61]:
image_files = natsorted(glob('human_in_the_loop/train/*.tif'))

width = 256
height = 256   

In [62]:
def mirror_half(image: np.ndarray, half: str) -> np.ndarray:
    if half == 'left':
        half = image[:, :width//2]
    elif half == 'right':
        half = image[:, width//2:]
    else:
        raise ValueError('Invalid half. Choose from "left" or "right".')
        
    augmented_image = np.concatenate((half, cv2.flip(half, 1)), axis=1)

    return augmented_image

In [52]:
import tifffile

for image_file in image_files:
    image_path = image_file
    image = tifffile.imread(image_path)
    
    for half in ['left', 'right']:
        augmented_image = mirror_half(image, half)
        output_file_path = os.path.join(output_path, f'{half}_mirrored_{os.path.basename(image_file)}')
        # Convert the image to BGR color space before saving
        tifffile.imwrite(output_file_path, augmented_image)


In [68]:
from glob import glob
from natsort import natsorted

labels_list = natsorted(glob('human_in_the_loop/test/*.npy'))
labels = []

for lbl in labels_list:
    labels.append(np.load(lbl, allow_pickle=True).item())

In [64]:
labels[0]['outlines']

array([[ 0,  0,  0, ...,  0,  0,  0],
       [ 0,  1,  1, ...,  0,  0,  0],
       [ 0,  1,  0, ...,  0,  0,  0],
       ...,
       [ 0,  0,  0, ...,  0,  0, 80],
       [ 0,  0,  0, ...,  0,  0, 80],
       [ 0,  0,  0, ..., 80, 80, 80]], dtype=uint16)

In [69]:
labell = labels

In [70]:
import tifffile

for i, image_file in enumerate(labels_list):
    image_path = image_file
    #image = tifffile.imread(image_path)
    image = labels[i]['outlines']
    
    for half in ['left', 'right']:
        augmented_image = mirror_half(image, half)
        labell[i]['outlines'] = augmented_image
        
        output_file_path = os.path.join(output_path, f'{half}_mirrored_{os.path.basename(image_file)}')
        # Convert the image to BGR color space before saving
        np.save(output_file_path, labell[i], allow_pickle=True)

In [22]:
import numpy as np

def mirror_half(label_path: str, half: str) -> np.ndarray:
    label = np.load(label_path, allow_pickle=True)

    if half == 'left':
        half = label[:, :width//2]
    elif half == 'right':
        half = label[:, width//2:]
    else:
        raise ValueError('Invalid half. Choose from "left" or "right".')

    augmented_label = np.concatenate((half, np.flip(half, 1)), axis=1)

    return augmented_label

In [23]:
for label_file in labels_list:
    label_path = label_file
    
    for half in ['left', 'right']:
        augmented_label = mirror_half(label_path, half)
        output_file_path = os.path.join(output_path, f'{half}_mirrored_{label_file}')
        np.save(output_file_path, augmented_label)

IndexError: too many indices for array: array is 0-dimensional, but 2 were indexed

In [9]:
def mirror_corner(image: np.ndarray, corner: str) -> np.ndarray:
    if corner == 'upper_left':
        selected_corner = image[:height//2, :width//2]
    elif corner == 'upper_right':
        selected_corner = image[:height//2, width//2:]
    elif corner == 'lower_left':
        selected_corner = image[height//2:, :width//2]
    elif corner == 'lower_right':
        selected_corner = image[height//2:, width//2:]
    else:
        raise ValueError('Invalid corner. Choose from "upper_left", "upper_right", "lower_left", "lower_right".')

    augmented_image = np.concatenate((selected_corner, cv2.flip(selected_corner, 1)), axis=1)
    augmented_image = np.concatenate((augmented_image, cv2.flip(augmented_image, 0)), axis=0)
    
    return augmented_image

In [10]:
for image_file in image_files:
    image_path = os.path.join(test_images_path, image_file)
    image = cv2.imread(image_path)
    
    for corner in ['upper_left', 'upper_right', 'lower_left', 'lower_right']:
        augmented_image = mirror_corner(image, corner)
        output_file_path = os.path.join(output_path, f'{corner}_mirrored_{image_file}')
        cv2.imwrite(output_file_path, augmented_image)

In [16]:
def add_gaussian_noise(image: np.ndarray, mean: float = 0.0, std_dev: float = 0.2) -> np.ndarray:
    noise = np.random.normal(mean, std_dev, image.shape).astype(np.uint8)
    noisy_image = cv2.add(image, noise)
    return noisy_image

In [12]:
def change_brightness_contrast(image: np.ndarray, brightness: int = 50, contrast: int = 50) -> np.ndarray:
    new_image = np.zeros(image.shape, image.dtype)
    alpha = 1.0 + contrast / 100.0
    beta = brightness
    new_image = cv2.convertScaleAbs(image, alpha=alpha, beta=beta)
    return new_image

In [25]:
def random_scale(image, scale_range):
    scale = np.random.uniform(scale_range[0], scale_range[1])
    new_size = (int(image.shape[1] * scale), int(image.shape[0] * scale))

    new_size = (min(new_size[0], image.shape[1]), min(new_size[1], image.shape[0]))

    x = np.random.randint(0, image.shape[1] - new_size[0] + 1)
    y = np.random.randint(0, image.shape[0] - new_size[1] + 1)

    # Crop the image to the new size and scale it back to the original size
    cropped = image[y:y + new_size[1], x:x + new_size[0]]
    scaled_image = cv2.resize(cropped, (image.shape[1], image.shape[0]))

    return scaled_image

In [26]:
for image_file in image_files:
    image_path = os.path.join(test_images_path, image_file)
    image = cv2.imread(image_path)
    
    # Add Gaussian noise
    noisy_image = add_gaussian_noise(image)
    output_file_path = os.path.join(output_path, f'noisy_{image_file}')
    cv2.imwrite(output_file_path, noisy_image)
    
    # Change brightness and contrast
    adjusted_image = change_brightness_contrast(image)
    output_file_path = os.path.join(output_path, f'adjusted_{image_file}')
    cv2.imwrite(output_file_path, adjusted_image)
    
    # Randomly scale the image
    scaled_image = random_scale(image, scale_range=(0.5, 1.5))
    output_file_path = os.path.join(output_path, f'scaled_{image_file}')
    cv2.imwrite(output_file_path, scaled_image)